### < Topic >

1. review
    
   1.1  카운터 기반 단어 표현    
      - BoW:하나의 문서에 대한 빈도수로 단어 표현( vector 형태, 1차원 )
      - DTM 여러 문서에 대한 빈도수로 단어 표현( matrix 형태, 2차원 )
      - TF-IDF: DTM에 가중치를 적용해서 단어 표현
      
   1.2  문서 유사도     
      - 코사인 유사도: 문서의 방향 판단
      - 유클리드 거리: 문서의 거리 판단
      - 자카드 유사도: 합집합에서 교집합의 비율 


2. topic
  
  2.1  토픽 모델링 (Topic Modeling)

## 유사도를 이용한 추천 시스템 구현하기

- TF-IDF와 코사인 유사도로 영화의 줄거리(overview data)에 기반하여 영화 추천

In [2]:
import pandas as pd

data = pd.read_csv('movies_metadata.csv')

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
data = data.head(20000)

In [6]:
data.isnull().sum()

adult                        0
belongs_to_collection    17601
budget                       0
genres                       0
homepage                 16945
id                           0
imdb_id                      7
original_language            1
original_title               0
overview                   135
popularity                   2
poster_path                 93
production_companies         1
production_countries         1
release_date                17
revenue                      2
runtime                     29
spoken_languages             2
status                      21
tagline                   8294
title                        2
video                        2
vote_average                 2
vote_count                   2
dtype: int64

In [7]:
# null값 제거

data['overview'] = data['overview'].fillna('')

In [13]:
# TF-IDF 구현

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english') # 불용어 처리 포함 객체 생성
tfidf_matrix = tfidf.fit_transform(data['overview']) 

print(tfidf_matrix.shape)

(20000, 47487)


In [15]:
# 코사인 유사도 계산
# 데이터의 방향(+,0,-) 비교

from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) # 영화들 간 유사도 계산

In [18]:
# 영화 타이틀과 인덱스를 가진 테이블

indices = pd.Series(data.index, index = data['title']).drop_duplicates()

print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [20]:
# 타이틀 입력 시 인덱스 값 리턴되는지 확인

idx = indices['Father of the Bride Part II']

print(idx)

4


In [26]:
# 선택한 영화에 대한 추천을 수행하는 함수

def get_recommendation(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))  # 모든 영화에 대한 해당 영화와의 유사도 파악
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True) # 유사도에 따른 영화 정렬
    sim_scores = sim_scores[1:11]   # 가장 유사한 영화 10개 추출
    movie_indices = [i[0] for i in sim_scores]  # 가장 유사한 영화 10개의 인덱스 추출
    
    return data['title'].iloc[movie_indices]

In [27]:
get_recommendation('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

In [28]:
get_recommendation('Toy Story')

15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
11399    For Your Consideration
1932                  Condorman
3057            Man on the Moon
485                      Malice
11606              Factory Girl
Name: title, dtype: object

In [56]:
# 오류 발생 - 'Batman'이 2개

get_recommendation('Batman')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [37]:
idx = indices['Batman']

print(idx)

Batman     585
Batman    8603
dtype: int64


In [46]:
data['title'].iloc[585]

'Batman'

In [47]:
data['title'].iloc[8603]

'Batman'

# Topic Modeling

- 의미 구조를 발견하는 Text Mining 기법
- BoW, DTM, TF-IDF는 단어의 빈도수를 이용한 수치화, 의미를 고려하지 못한다는 단점
- LSA (Latent Semantic Analysis), LDA ((Latent Dirichlet Allocation): DTM의 잠재된(Latent) 의미를 이끌어내는 방법


⇒ 검색 엔진, 고객 민원 시스템 등과 같이 문서 주제를 알아내는 일에 사용

## - LSA (Latent Semantic Analysis)
### : 차원 축소 기법 중 하나인 특이값 분해(SVD, Singular Value Decomposition)을 이용해 차원을 축소시키고, 단어들의 의미를 이끌어내는 알고리즘

<img src = "https://raw.githubusercontent.com/angeloyeo/angeloyeo.github.io/master/pics/2019-08-01_SVD/pic_SVD_restore.png">

<img src = "https://miro.medium.com/max/690/1*vg7UXvpptWGhNYvFy2WyTQ.png">

### - Full SVD

In [72]:
# DTM 생성

import numpy as np

A = np.array([[0,0,0,1,0,1,1,0,0], 
              [0,0,0,1,1,0,1,0,0], 
              [0,1,1,0,2,0,0,0,0], 
              [1,0,0,0,0,0,0,1,1]])
np.shape(A)

(4, 9)

In [73]:
# full SVD

U, s, VT = np.linalg.svd(A, full_matrices = True)

In [74]:
# U: mxm 직교 행렬

print(U.round(2))
np.shape(U)

[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]


(4, 4)

In [75]:
# s: mxn 대각 행렬
# linalg.svd()는 특이값 분해의 결과로 대각 행렬이 아니라 특이값의 리스트를 반환

print(s.round(2))
np.shape(s)

[2.69 2.05 1.73 0.77]


(4,)

In [76]:
# S: 행렬 모양 확인

S = np.zeros((4,9))  # 대각 행렬의 크기인 4 x 9의 임의의 행렬 생성
S[:4, :4] = np.diag(s)  # 특이값을 대각행렬에 삽입
print(S.round(2))
np.shape(S)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


(4, 9)

In [77]:
# VT: nxn 직교 행렬

print(VT.round(2))
np.shape(VT)

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]


(9, 9)

    U × S × VT = 기존 행렬

In [78]:
# allclose(): 2개의 행렬이 동일한지 확인

np.allclose(A, np.dot(np.dot(U,S), VT).round(2))

True

### - 절단된 SVD (Truncated SVD)
    t값(구하려는 topic 수)를 정하고, t값에 따른 절단된 SVD를 구성
    (하이퍼파라미터, 임의로 설정)

<img src = "https://wikidocs.net/images/page/24949/svd%EC%99%80truncatedsvd.PNG">

In [79]:
# 열 축소 (문서의 수 x 토픽의 수 t)

U = U[:, :2]
print(U.round(2))

[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


In [80]:
# 토픽의 수 t = 2

S = S[:2, :2]
print(S.round(2))

[[2.69 0.  ]
 [0.   2.05]]


In [81]:
# 행 축소 (토픽의 수 t x 단어의 수)

VT = VT[:2, :]
print(VT.round(2))

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


     U × S × VT = 기존 행렬

In [83]:
A_prime = np.dot(np.dot(U,S), VT)

print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


- U: 4 x 2의 크기를 가진 문서 벡터 - 문서 개수 x 토픽 수( t )

- VT: 2 X 9의 크기를 가진 단어 벡터 - 토픽 수( t ) x 단어 개수

    t: 의미 파악을 원하는 토픽 개수

## LSA 실습

문서 개수를 원하는 토픽의 개수로 압축한 뒤, 각 토픽당 가장 중요한 단어 5개를 출력하는 토픽 모델링 실습

#### 1. 데이터 수집

In [85]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers',
                                                                    'footers',
                                                                    'quotes'))
documents = dataset.data
len(documents)

11314

In [86]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [87]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


#### 2. 텍스트 전처리

In [89]:
news_df = pd.DataFrame({'document':documents})

# 특수문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() 
                                                                      if len(w) > 3]))
# 전체 단어 소문자로 변환
news_df['clean_doc'] = news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())


In [90]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [95]:
# 불용어 처리

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [96]:
tokenized_doc[1]

['yeah',
 'expect',
 'people',
 'read',
 'actually',
 'accept',
 'hard',
 'atheism',
 'need',
 'little',
 'leap',
 'faith',
 'jimmy',
 'logic',
 'runs',
 'steam',
 'sorry',
 'pity',
 'sorry',
 'feelings',
 'denial',
 'faith',
 'need',
 'well',
 'pretend',
 'happily',
 'ever',
 'anyway',
 'maybe',
 'start',
 'newsgroup',
 'atheist',
 'hard',
 'bummin',
 'much',
 'forget',
 'flintstone',
 'chewables',
 'bake',
 'timmons']

#### 3. TF-IDF 행렬 만들기

In [97]:
# 역토큰화

detokenized_doc = []

for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [98]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
                max_features= 1000,  # 상위 1,000개의 단어를 보존 
                max_df = 0.5, 
                smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape  # TF-IDF 행렬의 크기 확인

(11314, 1000)

#### 4. Topic Modeling

In [101]:
from sklearn.decomposition import TruncatedSVD

# 원본 데이터는 20개의 뉴스 카테고리를 갖고 있음 
svd_model = TruncatedSVD(n_components = 20, algorithm = 'randomized',
                         n_iter = 100, random_state = 122)
svd_model.fit(X)
len(svd_model.components_)  # LSA의 VT(단어 벡터)

20

In [102]:
np.shape(svd_model.components_)

(20, 1000)

    토픽의 수 t x 단어의 수 

In [103]:
# 가장 값이 큰 5개의 값을 찾아 단어 출력

terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):   # components - 단어 벡터
        print("Topic %d:" % (idx + 1), [(feature_names[i], topic[i].round(5)) 
                                        for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_, terms)

Topic 1: [('like', 0.21386), ('know', 0.20046), ('people', 0.19293), ('think', 0.17805), ('good', 0.15128)]
Topic 2: [('thanks', 0.32888), ('windows', 0.29088), ('card', 0.18069), ('drive', 0.17455), ('mail', 0.15111)]
Topic 3: [('game', 0.37064), ('team', 0.32443), ('year', 0.28154), ('games', 0.2537), ('season', 0.18419)]
Topic 4: [('drive', 0.53324), ('scsi', 0.20165), ('hard', 0.15628), ('disk', 0.15578), ('card', 0.13994)]
Topic 5: [('windows', 0.40399), ('file', 0.25436), ('window', 0.18044), ('files', 0.16078), ('program', 0.13894)]
Topic 6: [('chip', 0.16114), ('government', 0.16009), ('mail', 0.15625), ('space', 0.1507), ('information', 0.13562)]
Topic 7: [('like', 0.67086), ('bike', 0.14236), ('chip', 0.11169), ('know', 0.11139), ('sounds', 0.10371)]
Topic 8: [('card', 0.46633), ('video', 0.22137), ('sale', 0.21266), ('monitor', 0.15463), ('offer', 0.14643)]
Topic 9: [('know', 0.46047), ('card', 0.33605), ('chip', 0.17558), ('government', 0.1522), ('video', 0.14356)]
Topic 10

    LSA의 한계: 원본 데이터가 변경되면, 처음부터 다시 계산해야 함

## - LDA (Latent Dirichlet Analysis)
### : 문서들은 topic의 혼합으로 구성되어 있으며, topic들은 확률 분포에 기반하여 단어를 생성한다고 가정

- LSA: DTM을 차원 축소하여,축소 차원에서 근접 단어들을 topic으로 묶음
- LDA: 단어가 특정 topic에 존재할 확률과 문서에 특정 topic이 존재할 확률을 결합확률로 추정하여 topic 추출 (문서 > 토픽 > 단어)

    1) k값(구하려는 topic 수) 결정
     -> 하이퍼파라미터, 임의로 설정
    
    2) 모든 문서를 k개 중 하나의 topic에 할당
    
    <각 문서의 topic 분포>
    - 문서1 : 토픽 A 100%
    - 문서2 : 토픽 B 100%
    - 문서3 : 토픽 B 60%, 토픽 A 40%
    
    <각 topic의 단어 분포>
    - 토픽A : 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%
    - 토픽B : 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%
    
    3) 모든 할당이 완료될 때까지 아래 기준에 따라 반복 
    - p(topic t | document d): 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율
    - p(word w | topic t): 단어 w를 갖고 있는 모든 문서들 중 토픽 t가 할당된 비율

참조: https://lettier.com/projects/lda-topic-modeling/

-> DTM, LDA 수행해주는 사이트

## LDA 실습

In [104]:
# 위에서 전처리된 문서

tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [110]:
# 정수 인코딩
# gensim: 단어 빈도수를 쉽게 구할 수 있는 패키지

from gensim import corpora

In [111]:
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [112]:
print(dictionary[52])

well


In [113]:
len(dictionary)

64281

In [118]:
# LDA를 이용한 Topic Modeling

import gensim

NUM_TOPICS = 20  # 토픽 개수

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, 
                                           id2word=dictionary, passes=15) # passes: 알고리즘 동작 횟수
topics = ldamodel.print_topics(num_words=4)  # 4개의 단어만 출력 / 안 주면 10개 출력

for topic in topics:
    print(topic)

(0, '0.028*"entries" + 0.028*"printf" + 0.023*"title" + 0.012*"bytes"')
(1, '0.013*"food" + 0.008*"weaver" + 0.007*"fallacy" + 0.007*"exhaust"')
(2, '0.009*"people" + 0.008*"israel" + 0.007*"jews" + 0.007*"armenian"')
(3, '0.010*"government" + 0.010*"public" + 0.009*"encryption" + 0.008*"system"')
(4, '0.016*"period" + 0.013*"play" + 0.010*"power" + 0.009*"pittsburgh"')
(5, '0.014*"mail" + 0.012*"information" + 0.012*"available" + 0.011*"software"')
(6, '0.028*"output" + 0.023*"entry" + 0.014*"program" + 0.014*"line"')
(7, '0.035*"space" + 0.012*"nasa" + 0.007*"launch" + 0.007*"earth"')
(8, '0.012*"pens" + 0.009*"caps" + 0.009*"leafs" + 0.009*"hawks"')
(9, '0.034*"health" + 0.027*"medical" + 0.019*"disease" + 0.015*"patients"')
(10, '0.008*"ground" + 0.005*"wire" + 0.005*"bike" + 0.005*"picture"')
(11, '0.023*"sale" + 0.018*"offer" + 0.018*"price" + 0.018*"shipping"')
(12, '0.014*"would" + 0.013*"like" + 0.011*"know" + 0.008*"drive"')
(13, '0.017*"windows" + 0.015*"window" + 0.010*"usi

### 실습과제:

https://serieson.naver.com/movie/recommendList.nhn

- 위 사이트에서 영화별 줄거리를 크롤링해 파일에 저장하고, DTM/TF-IDF, 문서 유사도, Topic Modeling 적용하기

- 특히 영화 추천 실습 내용을 적용하기